# Loading and splitting Dataset
The goal of this notebook is to continue the development of the DeepTropism model using a Pytorch.<br>
The dataset was already created on the previous notebook and all the HIV-1 env V3 loop sequences where aligned.


In [2]:
# Libraries used on the analysis
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import random

In [ ]:
First we load the Dataframe with all the sequences published on referenced articles.<br>
The D


In [7]:
df = pd.read_csv('/home/gabriel/Documents/Repos/DeepTropism/dataset_unique_seqs_aligned_gapopen_15_old.tsv', 
                 sep='\t', names=['name', 'dataset', 'label','sequence'])
df.head()

,name,dataset,label,sequence
0,CCR5_1471_29187_CN_2003_B,geno2pheno,CCR5,CTQTQQQY-KKKY---T----------SRTR-ASM-----V-CNRR...
1,CCR5/CXCR4/CCR1/CCR2b/CCR3/CCR4_MVP5180_67_CM_...,geno2pheno,R5X4,CIREGIAE-VQDI---Y--T--G-P-----M-RWRSMTLKR-SNNT...
2,RKF859742,newdb,CCR5,CERPTMDI-QDIH------I--G-P-----M-AWYSTYIER-QAKG...
3,RAF009608,hivcopred,CCR5,CSRPEMDV-QEIR---N-----G-P-----M-AWYSMALAK-GGTT...
4,RKF859743,newdb,CCR5,CRRPAMKV-QEMR---I----------G--PMAWY-----S-MALE...


In [11]:
df = df[df.label != 'validation']
df.shape

(3572, 4)

In [47]:
# Function to call labels
def tropism_label(row):
    # For CCR5
    if row.label == 'CCR5':
        return 0
    # For CXCR4
    elif row.label == 'CXCR4':
        return 1
    # For R5X4
    elif row.label == 'R5X4':
        return 1

In [48]:
df['label_numeric'] = df.apply(tropism_label, axis=1)

In [49]:
df.label.value_counts()

CCR5     2757
R5X4      484
CXCR4     331
Name: label, dtype: int64

In [50]:
df.label_numeric.value_counts()

0    2757
1     815
Name: label_numeric, dtype: int64

# Spliting the Dataset for Cross Validation
We are going to create indices and set it to variables to make our cross validation reproducible. Our dataset is going to consist on:<br>
* Trainning = 80 %
* Validation = 10 %
* Test = 10 %

In [37]:
# Create a list of indices and shuffle it using seed
random.seed(23)
size = df.shape[0]
list_indices = list(range(size))
random.shuffle(list_indices)

In [41]:
# Now create the list of indices for trainning, validation and test
test_indices = list_indices[:int(size/10)]
train_val_indices = list_indices[int(size/10):]

In [40]:
len(test_indices)

357

In [42]:
len(train_val_indices)

3215

In [46]:
assert len(train_val_indices) + len(test_indices) == len(list_indices), "Splitting indices with error"

## Creating the Dataloaders for trainning

In [51]:
def get_array_from_sequence(protein_sequence):
    """
    Function to convert a protein sequence into a tensor.
    Each amino acid is represented by an numpy array of zeros of size 26,
    and the dict_aa_pos defines the position to be converted to 1.
    
    The function iterates over the protein sequences and stacks the arrays.
    At the end the arrays are linearized and converted to a tensor of size
    n x 26, with n the size of the protein.
    
    If the character is not present on the dict_aa_pos (eg. '-') the respective
    array is formed by zeros, and represents a missing value.
    """
    dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}
    
    f_array = np.zeros(26)
    for aa in protein_sequence:
        arr = np.zeros(26)
        if dict_aa_pos.get(aa):
            arr[dict_aa_pos.get(aa)] = 1
        f_array = np.vstack((f_array, arr))
    f_array = np.delete(f_array, 0,0)
    
    #return torch.from_numpy((f_array.flatten()).astype(float))
    return f_array.flatten().astype(float)

In [52]:
# Create list to append data from the df
list_data = []
list_labels = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df.iterrows():
    list_data.append(get_array_from_sequence(str(row.sequence)))
    list_labels.append(int(row.label_numeric))

In [56]:
len(list_data) == len(list_labels)

True

In [60]:
# For Test set
test_data = []
test_label = []
for j in test_indices:
    test_data.append(list_data[i])
    test_label.append(np.array(list_labels[i]))

test_tensor_x = torch.stack([torch.from_numpy(i) for i in test_data]) # transform to torch tensors
test_tensor_y = torch.stack([torch.from_numpy(i) for i in test_label])

test_dataset = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y) # create your test dataset
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64) # create your dataloader
